In [20]:
## 
import requests
import string
import pandas as pd

SET_META_URL_PREFIX = 'https://www.set.or.th/set/commonslookup.do?language=th&country=TH&prefix='
OUTPUT_FILE = '../data/meta/thai_stocks.csv'

prefix_list = ["NUMBER"]+list(string.ascii_uppercase[:27])
stock_table = pd.DataFrame()
for prefix in prefix_list:
    stock_table = stock_table.append(pd.read_html(SET_META_URL_PREFIX+prefix)[0],ignore_index=True)
    
stock_table.columns = ['ticker','name','market']
stock_table.set_index('ticker',inplace=True)

#print result
print('Print thai stock metadata headers ...')
print(stock_table.head())
print('Saving result to ',OUTPUT_FILE)

#Save result
stock_table.to_csv(OUTPUT_FILE)

Print thai stock metadata headers ...
                                                     name market
ticker                                                          
2S                       บริษัท 2 เอส เมทัล จำกัด (มหาชน)    mai
A                บริษัท อารียา พรอพเพอร์ตี้ จำกัด (มหาชน)    SET
AAV                บริษัท เอเชีย เอวิเอชั่น จำกัด (มหาชน)    SET
ABICO              บริษัท เอบิโก้ โฮลดิ้งส์ จำกัด (มหาชน)    mai
ABPIF   กองทุนรวมโครงสร้างพื้นฐานโรงไฟฟ้า อมตะ บี.กริม...    SET
Saving result to  ../data/meta/thai_stocks.csv


In [ ]:
import sys
import pandas as pd
import numpy as np
import time
sys.path.append('../')
import potzy
import potzy.siamchartdatareader as scdr

#LOG IN INTO SIAMCHART
dr=scdr.SiamChartDataReader()
dr.loginToSiamChart()

all_thai_tickers = potzy.metadata.getAllThaiTickers()

num_round=0
today = dt.date.today()
year,month,day=today.year, today.month, today.day
prefix_fs = '../data/siamchart/fs/'+'/'.join([str(year),str(month),str(day)])
prefix_div = '../data/siamchart/div/'+'/'.join([str(year),str(month),str(day)])
potzy.helpers.ensure_dir(prefix_fs)
potzy.helpers.ensure_dir(prefix_div)

while len(tickers_left) > 0 and num_round < 10:
    for ticker in tickers_left:
        try:
            tryRelogin=False
            while True:
                print('Loading data for ',ticker)
                df_fs=dr.getFinancialData(ticker)
                df_div=dr.getDividendData(ticker)
                print('Done for ticker ',ticker,' size of dividend and fs df are ',len(df_div),len(df_fs))
                if len(df_fs) <= 5 and not tryRelogin:
                    try:
                        dr.loginToSiamChart()
                    except:
                        print('log in not successful, probably stil in login session')
                    tryRelogin=True
                    time.sleep(1)
                    print(ticker, ": relogin and reload the data ... ")
                else:
                    break

            #reorganize columns
            prefix_cols=['revenue','net_profit','eps','dps','gross_profit','ebit','ebitda']
            cols=prefix_cols+[x for x in df_fs.columns if x not in prefix_cols]
            df_fs=df_fs[cols]

            #save result
            #df_fs.to_csv(prefix_fs+ticker+'.csv')
            #df_div.to_csv(prefix_div+ticker+'.csv')

            #
            tickers_loaded.add(ticker)

        except:
            #raise
            print('Error loading information for ticker ',ticker)
            bad_tickers.add(ticker)

    tickers_left = tickers_left.difference(tickers_loaded)
    num_round+=1
